In [1]:
from ml_in_finance_ensae.data import load_ff25_and_rf
ff25_excess, rf = load_ff25_and_rf()
ff25_excess.shape, rf.shape

((1194, 25), (1194,))

In [2]:
ff25_excess.head()
ff25_excess.describe().loc[["mean","std"]]

,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,ME2 BM5,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
mean,0.005679,0.006695,0.009790,0.011257,0.013392,0.006471,0.009319,0.009627,0.010308,0.012256,...,0.007263,0.007763,0.008417,0.009662,0.010271,0.006927,0.006493,0.007106,0.006702,0.009721
std,0.118947,0.096358,0.088333,0.082611,0.091465,0.079545,0.074647,0.072056,0.074198,0.086552,...,0.061807,0.060871,0.062839,0.068074,0.085432,0.053310,0.052435,0.055654,0.065171,0.084427


In [3]:
ff25_excess.mean().sort_values().head()
ff25_excess.mean().sort_values().tail()

ME2 BM4       0.010308
ME3 BM5       0.011046
ME1 BM4       0.011257
ME2 BM5       0.012256
SMALL HiBM    0.013392
dtype: float64

In [4]:
ff25_excess.abs().max().sort_values().tail(10)

ME2 BM2       0.780988
ME2 BM4       0.800670
ME1 BM3       0.828525
ME4 BM5       0.865740
ME2 BM5       0.899424
BIG HiBM      0.980192
SMALL HiBM    0.994873
ME1 BM4       1.048198
ME1 BM2       1.260068
SMALL LoBM    1.478201
dtype: float64

In [5]:
(ff25_excess.abs() > 1).sum().sort_values().tail(10)

ME4 BM2       0
BIG HiBM      0
ME5 BM2       0
ME5 BM3       0
ME4 BM5       0
BIG LoBM      0
ME5 BM4       0
ME1 BM2       1
ME1 BM4       1
SMALL LoBM    2
dtype: int64

In [6]:
mask = (ff25_excess.abs() > 1).any(axis=1)
ff25_excess.loc[mask].head()

,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,ME2 BM5,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
Date,,,,,,,,,,,,,,,,,,,,,
1932-07-01,1.478201,0.143031,0.417070,0.382666,0.540313,0.375574,0.276458,0.415467,0.543974,0.694952,...,0.308700,0.337641,0.359864,0.689068,0.800158,0.265680,0.372318,0.502583,0.525702,0.980192
1933-05-01,0.731695,1.260068,0.684879,1.048198,0.941436,0.764562,0.780988,0.754724,0.800670,0.571093,...,0.226143,0.308985,0.362391,0.400148,0.628022,0.172450,0.143758,0.212097,0.323279,0.249600
1939-09-01,1.134371,0.405210,0.671974,0.503550,0.859873,0.406774,0.388562,0.424786,0.390371,0.576113,...,0.154422,0.212196,0.214004,0.300451,0.369725,0.126583,0.113342,0.141924,0.322650,0.539351


In [7]:
ff25_excess.index[:5]


DatetimeIndex(['1926-07-01', '1926-08-01', '1926-09-01', '1926-10-01',
               '1926-11-01'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [4]:
ff25_excess.index.to_series().diff().value_counts().head()

Unnamed: 0
31 days    692
30 days    396
28 days     74
29 days     25
Name: count, dtype: int64

In [11]:
ff25_excess.std().median()

np.float64(0.07336039409061909)

In [12]:
ff25_excess.mean().median()

np.float64(0.009096731155778895)

In [9]:
ff25_excess.iloc[0].sort_values()

ME2 BM4      -0.028282
ME1 BM4      -0.023677
ME1 BM2      -0.019206
ME3 BM5      -0.014829
ME2 BM5      -0.006727
ME3 BM3      -0.006599
ME4 BM4       0.000565
ME1 BM3       0.002918
BIG HiBM      0.003423
ME2 BM3       0.003856
ME2 BM1       0.009918
ME4 BM3       0.011189
ME4 BM1       0.013176
ME4 BM2       0.013260
ME3 BM1       0.016871
SMALL HiBM    0.017383
ME5 BM3       0.018085
ME2 BM2       0.021907
ME3 BM2       0.022291
ME4 BM5       0.022478
ME5 BM4       0.029063
BIG LoBM      0.031048
ME3 BM4       0.035089
SMALL LoBM    0.056076
ME5 BM2       0.058709
Name: 1926-07-01 00:00:00, dtype: float64

In [10]:
ff25_excess.iloc[0].describe()

count    25.000000
mean      0.011519
std       0.021753
min      -0.028282
25%       0.000565
50%       0.013176
75%       0.022291
max       0.058709
Name: 1926-07-01 00:00:00, dtype: float64